In [ ]:
import re
import pandas as pd
from collections import Counter
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from collections import defaultdict

from classes.utils import check_options, extract_option

BENCH =  "/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_"
BASE =  "/home/cc/PHD/HealthBranches/results/results_QUIZ_baseline_"
MEDQA =  "/home/cc/PHD/HealthBranches/results-medqa/results_QUIZ_medqa_"

In [ ]:
def change_line(df_old, df_new):
    # Itera sulle righe del secondo CSV
    for _, row in df_new.iterrows():
        idx = row["ID"]  # ID nel secondo CSV che corrisponde all'indice del primo CSV
        label = row["label"]
        
        if label == "1":
            df_old.at[idx, "correct_option"] = row["correct_option"]
        elif label == "2":
            df_old.at[idx, "question"] = row["question"]

    return df_old

def evaluate_answers(file_path, model):
    df = pd.read_csv(file_path)
    # subset_df = pd.read_csv("/home/cc/PHD/HealthBranches/question_checked.csv")

    #df = df[~df['question'].isin(subset_df['question'])] # senza le checked
    # df = df[df['question'].isin(subset_df['question'])] # solo checked

    df['correct_answer'] = df['real']
    accs = []

    for col in [c for c in df.columns if c.startswith(("zero_shot", "one_shot"))]:
        df[f'{col}_choice'] = df[col].apply(extract_option)
        df[f'{col}_is_correct'] = df[f'{col}_choice'] == df['correct_answer']
        accuracy = df[f'{col}_is_correct'].mean()
        print(f'Accuracy for {col}: {accuracy:.2%}')
        accs.append(accuracy)
    
    accs.insert(0, model)

    return accs

def evaluate_answers_by_cond(file_path, model, condition_value=None):
    df = pd.read_csv(file_path)
    
    # Filter the DataFrame for the specific condition if provided.
    if condition_value is not None:
        df = df[df['name'] == condition_value]
    
    # Set up the correct answer column.
    df['correct_answer'] = df['real']
    accs = []

    # Loop through each relevant column (those starting with "zero_shot" or "one_shot").
    for col in [c for c in df.columns if c.startswith(("zero_shot", "one_shot"))]:
        df[f'{col}_choice'] = df[col].apply(extract_option)
        df[f'{col}_is_correct'] = df[f'{col}_choice'] == df['correct_answer']
        accuracy = df[f'{col}_is_correct'].mean()
        # print(f'Accuracy for {col} under condition  == {condition_value}: {accuracy:.2%}')
        # if accuracy < 0.5:
        #      print(f'Accuracy for {col} under condition  == {condition_value}: {accuracy:.2%}')
        accs.append(accuracy)
    
    accs.insert(0, model)
    accs.insert(0, condition_value)
    return accs

def find_incorrect_indices(file_path, model):
    df = pd.read_csv(file_path)
    subset_df = pd.read_csv("/home/cc/PHD/HealthBranches/question_checked.csv")

    #df = df[~df['question'].isin(subset_df['question'])] # senza le checked
    # df = df[df['question'].isin(subset_df['question'])] # solo checked
    print(f"dataframe shape: {df.shape}, model: {model}")

    incorrect_indices = df.index[df["zero_shot"].apply(extract_option) != df["real"]].tolist()
    return incorrect_indices

def find_common_wrongs(lists_of_indices):
    if not lists_of_indices:
        return []
    
    # Start with the set of indices from the first list
    common = set(lists_of_indices[0])
    
    # Intersect with the indices from each subsequent list
    for indices in lists_of_indices[1:]:
        common &= set(indices)
    
    # Return the sorted list of common indices
    return sorted(common)

def create_folder_file_dict(root_folder):
    folder_dict = {}
    # List all items in the root folder
    for item in os.listdir(root_folder):
        item_path = os.path.join(root_folder, item)
        # Check if the item is a directory (folder)
        if os.path.isdir(item_path):
            # List all files in this subfolder
            files = os.listdir(item_path)
            # Optionally, filter out directories if needed:
            files = [f.lower().split('.')[0] for f in files if os.path.isfile(os.path.join(item_path, f))]
            folder_dict[item] = files
    return folder_dict

In [ ]:
df = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_bench_DeepSeek-R1-Distill-Qwen-7B.csv")

for row in df.iterrows():
    if extract_option(df["zero_shot_rag"]) == None:
        print(df["zero_shot"])

evaluate_answers(BENCH + "DeepSeek-R1-Distill-Qwen-7B.csv", "DeepSeek-R1-Distill-Qwen-7B")

In [ ]:
models = ["mistral:7b", "gemma:7b", "gemma2:9b", "gemma3:4b", "llama3.1:8b", "qwen2.5:7b", 
          "phi4:14b", "llama2:7b", "Llama-3.3-70B-Instruct-Turbo-Free"]
bench = [evaluate_answers(f"{BENCH}{model}.csv", model) for model in models]
baseline = [evaluate_answers(f"{BASE}{model}.csv", model) for model in models]

In [ ]:
conditions = pd.read_csv("/home/cc/PHD/HealthBranches/results/results_QUIZ_baseline_Llama-3.3-70B-Instruct-Turbo-Free.csv")['name'].unique().tolist()
cats = create_folder_file_dict("/home/cc/PHD/HealthBranches/new_trees")

res = []
for model in models:
    for condition in conditions:
        accs = evaluate_answers_by_cond(f"{BENCH}{model}.csv", model, condition)
        res.append(accs)

In [ ]:
file_to_folder = {}
for folder, files in cats.items():
    for file in files:
        file_to_folder[file] = folder

# Step 2: Replace the first element in each sublist with the corresponding folder name
for sublist in res:
    file_name = sublist[0]
    if file_name in file_to_folder:
        sublist[0] = file_to_folder[file_name]
    else:
        # Handle the case where the file name is not found in the dictionary
        sublist[0] = 'Unknown Folder'

In [ ]:
# Step 1: Aggregate values by category (removing model info)
category_values = defaultdict(list)

for item in res:
    category = item[0].replace("trees_", "")  # Remove "trees_" prefix
    value1 = float(item[2])  # First value column
    value2 = float(item[3])  # Second value column
    category_values[category].extend([value1])  # Store both values

# Compute mean for each category across all models
mean_values = {category: np.mean(values) for category, values in category_values.items()}

# Step 2: Extract names and mean values for plotting
names = list(mean_values.keys())
values = list(mean_values.values())

# Step 3: Create the horizontal bar chart
plt.figure(figsize=(12, 8))
bars = plt.barh(names, values, color='skyblue')

# Set x-axis limits for proper scaling
plt.xlim(0, max(values) + 0.1)

# Add value labels on bars
for bar, value in zip(bars, values):
    plt.text(value + 0.02, bar.get_y() + bar.get_height()/2, f"{value:.2f}", va='center')

# Labels and title
plt.xlabel('Mean Value (All Models)')
plt.ylabel('Category')
plt.title('Mean Values per Category Across All Models (zero shot)')

# Invert y-axis to keep the first item on top
plt.gca().invert_yaxis()
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Show plot
plt.show()

In [ ]:
models = ["mistral:7b", "gemma:7b", "gemma2:9b", "llama3.1:8b", "qwen2.5:7b", "phi4:14b"]
wrongs = [find_incorrect_indices(f"{BASE}{model}.csv", model) for model in models]
w2 = [find_incorrect_indices(f"{BENCH}{model}.csv", model) for model in models]

In [ ]:
len(find_common_wrongs(wrongs+w2))

In [ ]:
df = pd.read_csv(f"{BASE}{"gemma2:9b"}.csv")
subset_df = pd.read_csv("/home/cc/PHD/HealthBranches/question_checked.csv")

#df = df[~df['question'].isin(subset_df['question'])] # senza le checked
df = df[df['question'].isin(subset_df['question'])] # solo checked
filtered_df = df.loc[find_common_wrongs(wrongs+w2)]
print(f"filtered_df shape: {filtered_df.shape}")

# Save the filtered dataframe to a CSV file without writing the index column
# filtered_df.to_csv("/home/cc/PHD/HealthBranches/questions_to_check_BASE.csv", index=False)

In [ ]:
print("### BASELINE ###")
for model in models:
    print(f"Results for {model}")
    evaluate_answers(f"{BASE}{model}.csv", model)

In [ ]:
print("### BENCH ###")
for model in models:
    print(f"Results for {model}")
    evaluate_answers(f"{BENCH}{model}.csv", model)

In [ ]:
print("### MEDQA ###")
for model in models:
    print(f"Results for {model}")
    evaluate_answers(f"{MEDQA}{model}.csv", model)

In [ ]:
def show_chart(bench, baseline, bar1, bar2, bar3, xl, yl, title):
    # Creiamo un dizionario dalla lista baseline per una ricerca veloce
    baseline_dict = {item[0]: item[1] for item in baseline}

    # Uniamo le liste
    merged_list = [item + [baseline_dict[item[0]]] for item in bench if item[0] in baseline_dict]

    # Ordiniamo la lista in base al primo float (item[1])
    merged_list.sort(key=lambda x: x[1], reverse=True)

    print(merged_list)

    # Estrai le etichette e i valori
    labels = [x[0] for x in merged_list]
    values1 = [x[1] for x in merged_list]
    values2 = [x[2] for x in merged_list]
    values3 = [x[3] for x in merged_list]

    # Imposta la posizione delle barre con più spazio tra i gruppi
    x = np.arange(len(labels)) * 1.3  # Moltiplica per aumentare la distanza tra i gruppi
    width = 0.35  # Larghezza delle barre

    # Aumenta le dimensioni del grafico
    fig, ax = plt.subplots(figsize=(16, 10))
    bars1 = ax.bar(x - width, values1, width, label=bar1)
    bars2 = ax.bar(x, values2, width, label=bar2)
    bars3 = ax.bar(x + width, values3, width, label=bar3)    

    # Etichette e titolo
    ax.set_xlabel(xl)
    ax.set_ylabel(yl)
    ax.set_title(title)
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    # Mostra il grafico
    plt.show()

show_chart(bench, baseline, "no RAG", "RAG", "Baseline", "Models", "Accuracy", "Benchmark")

In [ ]:
import pandas as pd
import ast
import random
import string
from collections import Counter

def balance_correct_answer(csv_file, output_file):
    # Carica il file CSV
    df = pd.read_csv(csv_file)
    
    # Definisce le lettere per le opzioni (A, B, C, D, E)
    option_letters = list(string.ascii_uppercase[:5])
    
    # Conta la distribuzione attuale delle risposte corrette
    correct_counts = Counter(df['correct_option'])
    
    # Calcola il numero desiderato per ciascuna lettera
    target_count = len(df) // 5
    
    # Inizializza un dizionario per tenere traccia delle assegnazioni
    assigned_counts = {letter: 0 for letter in option_letters}
    
    # Funzione per ribilanciare la posizione della risposta corretta
    def rebalance(row):
        options = ast.literal_eval(row['options'].replace("['", '["').replace("']", '"]').replace("', '", '", "'))  # Converte la stringa in lista
        correct_letter = row['correct_option']
        correct_index = option_letters.index(correct_letter)
        correct_answer = options[correct_index]
        
        # Trova le lettere meno usate
        available_letters = [letter for letter in option_letters if assigned_counts[letter] < target_count]
        
        # Se tutte le lettere sono bilanciate, assegna a caso
        new_correct_letter = random.choice(available_letters) if available_letters else random.choice(option_letters)
        new_correct_index = option_letters.index(new_correct_letter)
        
        # Rimescola le risposte
        random.shuffle(options)
        
        # Sposta la risposta corretta nella nuova posizione
        options.remove(correct_answer)
        options.insert(new_correct_index, correct_answer)
        
        # Aggiorna il conteggio
        assigned_counts[new_correct_letter] += 1
        
        return pd.Series([str(options), new_correct_letter])
    
    # Applica la funzione a ogni riga
    df[['options', 'correct_option']] = df.apply(rebalance, axis=1)
    
    # Salva il nuovo file CSV
    df.to_csv(output_file, index=False)

    return df


df = pd.read_csv("/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full.csv")
print("Distribuzione prima:")
print(Counter(df["correct_option"]))

# # Esegui la funzione su un file di esempio
# bal_df = balance_correct_answer("/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full.csv", "/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full_balanced.csv")

# # df = balance_correct_options(df)
# print("Distribuzione dopo:")
# print(Counter(bal_df["correct_option"]))

# df.to_csv("/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full_balanced.csv", index=False)

In [ ]:
# Carica i CSV
df_dataset = pd.read_csv('/home/cc/PHD/HealthBranches/questions_pro/ultimate_questions_v3_full_balanced.csv')       # CSV completo delle domande
df_subset = pd.read_csv('/home/cc/PHD/HealthBranches/questions_to_check.csv')         # CSV con le domande da correggere
df_corrected = pd.read_csv('/home/cc/PHD/HealthBranches/question_checked.csv')    # CSV con le domande corrette; contiene la colonna "ID" (l'indice nel CSV subset)

# Resetta l'indice di df_subset per garantire che sia numerato da 0 a len(df_subset)-1
# df_subset.reset_index(drop=True, inplace=True)

mapping = {}
for _, row in df_corrected.iterrows():
    idx = int(row['ID'])
    if idx < len(df_subset):
        original_question = df_subset.loc[idx, 'question']
        mapping[original_question] = {
            'question': row['question'],
            'answer': row['answer'],
            'options': row['options'],
            'correct_option': row['correct_option'],
            'path': row['path']
        }
    else:
        print(f"Warning: ID {idx} non presente in df_subset")

# Funzione per aggiornare una riga del dataset, se la domanda è presente nel mapping
def update_row(row):
    corrections = mapping.get(row['question'])
    if corrections:
        row['question'] = corrections['question']
        row['answer'] = corrections['answer']
        row['options'] = corrections['options']
        row['correct_option'] = corrections['correct_option']
        row['path'] = corrections['path']
    return row

# Applica la funzione a ogni riga del dataset
df_dataset = df_dataset.apply(update_row, axis=1)
df_dataset.drop(columns=['answer'], inplace=True)

# Salva il dataset aggiornato
df_dataset.to_csv('dataset_updated.csv', index=False)
print("Dataset aggiornato salvato come 'dataset_updated.csv'")